
# LAB 10 – IMAGE COMPRESSION

**Tasks Covered**
1. Implement Huffman Coding  
2. Implement JPEG-like DCT Compression  
3. Compute CR, MSE, PSNR, RD  
4. Compare Original vs Compressed Images  

This notebook is self-contained and can be run step-by-step.


In [ ]:

# Imports
import numpy as np
import cv2
import matplotlib.pyplot as plt
from collections import Counter
import heapq
import math



## 1. Load and Display Image


In [ ]:

# Load grayscale image
image = cv2.imread('sample.jpg', cv2.IMREAD_GRAYSCALE)
if image is None:
    # create synthetic image if no file exists
    image = np.tile(np.arange(256, dtype=np.uint8), (256,1))

plt.imshow(image, cmap='gray')
plt.title("Original Image")
plt.axis('off')



## 2. Huffman Coding Implementation


In [ ]:

# Huffman Coding Implementation
class HuffmanNode:
    def __init__(self, symbol=None, freq=0):
        self.symbol = symbol
        self.freq = freq
        self.left = None
        self.right = None
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(data):
    freq = Counter(data.flatten())
    heap = [HuffmanNode(sym, fr) for sym, fr in freq.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        n1 = heapq.heappop(heap)
        n2 = heapq.heappop(heap)
        parent = HuffmanNode(None, n1.freq + n2.freq)
        parent.left = n1
        parent.right = n2
        heapq.heappush(heap, parent)

    return heap[0]

def generate_codes(node, prefix="", codebook={}):
    if node:
        if node.symbol is not None:
            codebook[node.symbol] = prefix
        generate_codes(node.left, prefix + "0", codebook)
        generate_codes(node.right, prefix + "1", codebook)
    return codebook

tree = build_huffman_tree(image)
codes = generate_codes(tree)

encoded_bits = sum(len(codes[p]) for p in image.flatten())
original_bits = image.size * 8
CR_huffman = original_bits / encoded_bits
CR_huffman



## 3. JPEG-like DCT Compression


In [ ]:

# DCT Compression
def block_dct(img, block=8):
    h, w = img.shape
    dct_img = np.zeros_like(img, dtype=float)
    for i in range(0, h, block):
        for j in range(0, w, block):
            dct_img[i:i+block, j:j+block] = cv2.dct(
                np.float32(img[i:i+block, j:j+block])
            )
    return dct_img

def block_idct(dct_img, block=8):
    h, w = dct_img.shape
    img = np.zeros_like(dct_img, dtype=float)
    for i in range(0, h, block):
        for j in range(0, w, block):
            img[i:i+block, j:j+block] = cv2.idct(
                np.float32(dct_img[i:i+block, j:j+block])
            )
    return img

# Quantization matrix
Q = np.ones((8,8)) * 20

dct_img = block_dct(image)
quantized = np.round(dct_img / Q)
reconstructed = block_idct(quantized * Q)
reconstructed = np.clip(reconstructed, 0, 255).astype(np.uint8)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.imshow(image, cmap='gray'); plt.title("Original")
plt.subplot(1,2,2); plt.imshow(reconstructed, cmap='gray'); plt.title("Compressed")
plt.axis('off')



## 4. Performance Metrics (CR, MSE, PSNR, RD)


In [ ]:

# Metrics
def mse(original, compressed):
    return np.mean((original - compressed) ** 2)

def psnr(original, compressed):
    m = mse(original, compressed)
    if m == 0:
        return float('inf')
    return 10 * math.log10((255**2) / m)

MSE = mse(image, reconstructed)
PSNR = psnr(image, reconstructed)

compressed_bits = np.count_nonzero(quantized) * 8
CR_dct = original_bits / compressed_bits
RD = compressed_bits / image.size

MSE, PSNR, CR_dct, RD



## 5. Comparison Summary

- Huffman Coding provides lossless compression.
- DCT Compression is lossy but achieves higher compression.
- Trade-off exists between image quality (PSNR) and compression ratio.
